In [4]:
import numpy as np

In [52]:
s0 = 100
r = 0.02594
q = 0
vol = 0.2803

In [38]:
dx = 0.01 * 100
h = dx
Nx = 500
h

1.0

In [15]:
m = 500 #가격 Mesh (세로축)
N = 50 #시간 Mesh (가로축)

In [14]:
Smax = m * dx
Smin = 0

Smin, Smax

(0, 500.0)

In [21]:
T = 3
N = 50
Nt = 6*N

dt = T/Nt
k = dt


In [23]:
cr = np.array([0.035, 0.07, 0.105, 0.14, 0.175, 0.21])
K = np.array([0.95, 0.90, 0.85, 0.80, 0.75, 0.75])
early_redemption = np.array([0.5, 1, 1.5, 2, 2.5, 3])

In [43]:
Barrier = 50
Rtag = 2 * early_redemption * N
s0tag = 0
Barriertag = 0

501

In [69]:
a, b, c, d = np.zeros(m-1), np.zeros(m-1), np.zeros(m-1), np.zeros(m-1), #Mesh 만들기
S = np.linspace(Smin, Smax, m+1) # 0 ~ 500 까지
x = S/s0 #기초자산 가격으로 Scaling

u = np.zeros((Nt+1, Nx+1))
w = np.zeros((Nt+1, Nx+1))

for i in range(Nx+1):
    if (i*dx <= s0):
        s0tag = i+1

    if (i*dx <= s0 * Barrier):
        Barriertag = i+1

a = k/2 * ((r-q) + (vol*S[1:-1]/h)**2)
b = k/4 * ((r*S[1:-1])/h - (vol*S[1:-1]/h)**2)
c = k/4 * (-(r*S[1:-1])/h - (vol*S[1:-1]/h)**2)

'''Barrier Term'''

for k in range(Nx+1):

    if (x[k] > Barrier):
        u[0, k] = ( 1 + cr[5] )
    
    else:
        u[0, k] = x[k]

    if (x[k] > K[5]):
        w[0, k] = (1 + cr[5])
    
    else:
        w[0, k] = x[k]

'''그냥'''

for i in range(1, Nt+1):

    d1 = u[i-1, 1:Nx]
    d1[0] = -b[0] * u[i,0] + ()

In [ ]:
def CNFDMf(PutCall,xmin,K,rf,q,sigma,dt,dx,N,M):
    xh = np.array(list(range(M+1)))                                         # number of node: M+1
    tk = np.array(list(range(N+1)))
    
    v = np.zeros((N+1,M+1))                                                 # solution (shape:(N+1,M+1))
    
    # BC at expiration (n=0)
    if PutCall == 'C':
        v[0,0:M+1] = np.maximum(xmin + xh*dx - K, 0)
    elif PutCall =='P':
        v[0,0:M+1] = np.maximum(K - (xmin + xh*dx), 0)
        
    # BC at x = Smax or Smin (m=M or m=0)
    if PutCall =='C':
        v[0:N+1,0] = 0                                                         # call value when underlying is 0
        v[0:N+1,M] = (xmin + M*dx)*np.exp(-q*tk*dt) - K*np.exp(-rf*tk*dt)      # call value when underlying is large 
    elif PutCall == 'P':
        v[0:N+1,0] = K*np.exp(-rf*tk*dt)                                        # put value when underlying is 0
        v[0:N+1,M] = 0                                                          # put value when underlying is large 

    
    # Coefficients of matrix for CNFDM
    a = (dt/2)*(rf + (sigma**2)*(xh**2))
    b = (dt/4)*((rf-q)*xh - (sigma**2)*(xh**2))
    c = -(dt/4)*((rf-q)*xh + (sigma**2)*(xh**2))

    # a-(b+c)>0, the matrix is strictly diagonally dominant
    d = np.zeros(M+1)
    
    for n in range (0,N):         
        d[1:M] = - np.multiply(b[1:M],v[n,0:M-1]) + np.multiply(1-a[1:M],v[n,1:M]) - np.multiply(c[1:M],v[n,2:M+1])
        d[1] = d[1] - b[1]*v[n+1,0]
        d[M-1] = d[M-1] - c[M-1]*v[n+1,M]       
        v[n+1,1:M] = Thomas_Algorithm(1 + a[1:M],b[1:M],c[1:M],d[1:M])

    price = np.zeros(M+1)
    price = v[N,0:M+1]
    return price


CNFDMf('P', 0, )